# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-vl38eml4
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-vl38eml4
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [5]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [6]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [7]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [8]:
torch.set_grad_enabled(True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
p = 113
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=128,
    n_heads=2,
    d_head=64,
    d_mlp=512,
    act_fn='relu',
    n_ctx=3, # a b =
    d_vocab=p+1, #0 1 ... p-1 =
    d_vocab_out=p,
    normalization_type=None,
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [11]:
# disable biases
for name, param in model.named_parameters():
    if 'b_' in name:
        param.requires_grad = False
    print(name, param.shape, param.requires_grad)

embed.W_E torch.Size([114, 128]) True
pos_embed.W_pos torch.Size([3, 128]) True
blocks.0.attn.W_Q torch.Size([2, 128, 64]) True
blocks.0.attn.W_K torch.Size([2, 128, 64]) True
blocks.0.attn.W_V torch.Size([2, 128, 64]) True
blocks.0.attn.W_O torch.Size([2, 64, 128]) True
blocks.0.attn.b_Q torch.Size([2, 64]) False
blocks.0.attn.b_K torch.Size([2, 64]) False
blocks.0.attn.b_V torch.Size([2, 64]) False
blocks.0.attn.b_O torch.Size([128]) False
blocks.0.mlp.W_in torch.Size([128, 512]) True
blocks.0.mlp.b_in torch.Size([512]) False
blocks.0.mlp.W_out torch.Size([512, 128]) True
blocks.0.mlp.b_out torch.Size([128]) False
unembed.W_U torch.Size([128, 113]) True
unembed.b_U torch.Size([113]) False


# Training

## Task dataset

In [12]:
a_vec = einops.repeat(torch.arange(p), "i -> (i j)", j=p)
print(a_vec.shape)
print(a_vec[:5])

b_vec = einops.repeat(torch.arange(p), "i -> (j i)", j=p)
print(b_vec.shape)
print(b_vec[:5])

eq_vec = torch.ones_like(a_vec) * (cfg.d_vocab-1)
print(eq_vec.shape)
print(eq_vec[:5])

dataset = torch.stack([a_vec, b_vec, eq_vec], dim=-1).to(device)
print(dataset.shape)
print(dataset[:5])

torch.Size([12769])
tensor([0, 0, 0, 0, 0])
torch.Size([12769])
tensor([0, 1, 2, 3, 4])
torch.Size([12769])
tensor([113, 113, 113, 113, 113])
torch.Size([12769, 3])
tensor([[  0,   0, 113],
        [  0,   1, 113],
        [  0,   2, 113],
        [  0,   3, 113],
        [  0,   4, 113]], device='cuda:0')


In [13]:
labels = (dataset[:, 0] - dataset[:, 1]) % p
print(labels.shape)
print(labels[:5])

torch.Size([12769])
tensor([  0, 112, 111, 110, 109], device='cuda:0')


In [14]:
DATA_SEED = 0
torch.manual_seed(DATA_SEED)

train_frac = 0.3

cutoff = int(train_frac * dataset.shape[0])
indices = torch.randperm(dataset.shape[0])

train_indices = indices[:cutoff]
train_data = dataset[train_indices]
train_labels = labels[train_indices]
print(train_data.shape)
print(train_data[:5])
print(train_labels.shape)
print(train_labels[:5])

test_indices = indices[cutoff:]
test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(test_data.shape)
print(test_data[:5])
print(test_labels.shape)
print(test_labels[:5])


torch.Size([3830, 3])
tensor([[ 37,   1, 113],
        [ 23,   9, 113],
        [  2,  43, 113],
        [  7,  34, 113],
        [ 42, 101, 113]], device='cuda:0')
torch.Size([3830])
tensor([36, 14, 72, 86, 54], device='cuda:0')
torch.Size([8939, 3])
tensor([[ 55,  55, 113],
        [ 69, 109, 113],
        [ 13,  17, 113],
        [103,  54, 113],
        [ 78, 112, 113]], device='cuda:0')
torch.Size([8939])
tensor([  0,  73, 109,  49,  79], device='cuda:0')


## Loss Fn

In [15]:
def loss_fn(logits, labels):
    if logits.ndim == 3:
        logits = logits[:, -1, :]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])
    return -correct_log_probs.mean()

with torch.no_grad():
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    print(train_loss)

    test_logits = model(test_data)
    test_loss = loss_fn(test_logits, test_labels)
    print(test_loss)

tensor(4.7344, device='cuda:0', dtype=torch.float64)
tensor(4.7328, device='cuda:0', dtype=torch.float64)


In [16]:
print("uniform loss:", np.log(cfg.d_vocab_out))

uniform loss: 4.727387818712341


## Setup optimizer

In [17]:
lr = 1e-3
wd = 1.0
betas = (0.9, 0.98)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=wd, betas=betas)

## Training Loop

In [18]:
num_epochs = 45000

train_losses = []
test_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    train_losses.append(train_loss.item())

    optimizer.step()
    optimizer.zero_grad()

    with torch.inference_mode():
        test_logits = model(test_data)
        test_loss = loss_fn(test_logits, test_labels)
        test_losses.append(test_loss.item())

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train_loss: {train_loss.item()}, test_loss: {test_loss.item()}")

  0%|          | 0/45000 [00:00<?, ?it/s]

Epoch: 0, train_loss: 4.7343881948589805, test_loss: 4.732498731317706
Epoch: 100, train_loss: 3.0468600966460557, test_loss: 7.732328339162696
Epoch: 200, train_loss: 0.07014745060754246, test_loss: 21.438972101643238
Epoch: 300, train_loss: 0.013783683764551389, test_loss: 23.473826549322585
Epoch: 400, train_loss: 0.0044287177325980235, test_loss: 24.837379525628595
Epoch: 500, train_loss: 0.0014660979381252227, test_loss: 26.38293485052988
Epoch: 600, train_loss: 0.000495598946032773, test_loss: 27.98224159299727
Epoch: 700, train_loss: 0.00016999042494836766, test_loss: 29.649385033709983
Epoch: 800, train_loss: 5.917667948273686e-05, test_loss: 31.353947280503395
Epoch: 900, train_loss: 2.092509336568305e-05, test_loss: 33.04546185049554
Epoch: 1000, train_loss: 7.642714716944083e-06, test_loss: 34.69023076744825
Epoch: 1100, train_loss: 2.9701557981174732e-06, test_loss: 36.20883369953911
Epoch: 1200, train_loss: 1.3027082035762467e-06, test_loss: 37.49172114940741
Epoch: 1300, 

In [19]:
lines(
    [train_losses, test_losses],
    title="Loss Curves",
    xaxis="Epoch", yaxis="Loss",
    labels=["Train", "Test"]
)

## Sanity check

In [20]:
with torch.inference_mode():
    test_logits = model(test_data)
    test_logits = test_logits[:, -1, :]
    preds = test_logits.argmax(dim=-1)
    test_acc = (preds == test_labels).float().mean()
    print("Test accuracy:", test_acc.item())

Test accuracy: 1.0


# Save model

In [21]:
%mkdir ../models

In [22]:
filename = "../models/modular_subtraction_model.pt"
torch.save(model.state_dict(), filename)

In [23]:
# sanity check we can load model
p = 113
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=128,
    n_heads=2,
    d_head=64,
    d_mlp=512,
    act_fn='relu',
    n_ctx=3, # a b =
    d_vocab=p+1, #0 1 ... p-1 =
    d_vocab_out=p,
    normalization_type=None,
    device=device,
    seed=0
)

model_loaded = HookedTransformer(cfg)
model_loaded.load_state_dict(torch.load(filename), strict=True)

<All keys matched successfully>